In [1]:
import warnings
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans

In [2]:
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True, precision=3)

### load data

In [3]:
data = np.loadtxt("../data/rin.dat")  #  data matrix

print("number of entities:", data.shape[0], ", number of features:", data.shape[1])

with open("../data/namrin", 'r') as fp:  # load names as list of string
    names_ = fp.readlines()

with open("../data/varrin", 'r') as fp:  # load features names as list of string
    features_ = fp.readlines()

number of entities: 91 , number of features: 5


In [4]:
def standardizer(x):
    
    """
        standardize entity-to-feature data matrix by 
          applying Z-scoring and Range standardization methods
        
        Arguments: 
            x, numpy array, entity-to-feature data matrix
        
        Returns:
            Z-scored and Range standardized data matrices
    """
    
    x_ave = np.mean(x, axis=0)
    x_rng = np.ptp(x, axis=0)
    x_std = np.std(x, axis=0)
    x_zscr_std = np.divide(np.subtract(x, x_ave), x_std)   # Z-scoring standardization
    x_rng_std = np.divide(np.subtract(x, x_ave), x_rng)  # Range standardization 
    return x_zscr_std, x_rng_std

In [5]:
def apply_kmeans(x_org, n_clusters, n_repeats, std_method='r'):


    """
        Calls Kmeans algorithm from Sklearn library.
        Parameters:
            x, a numpy arrary, entity-to-feature matrix,
            n_clusters, int, number of clusters to detect,
            n_repeats, int, number of repeats for different initilization
        Return:
            centroids, clusters labels over
    """
    
    tmp_inertia = 0
    clusters, best_clusters = {}, {}
    indices, best_indices = {}, {}
    cluster_means, best_cluster_means = {}, {}
    differences, best_differences = {}, {}
    rel_differences, best_rel_differences = {}, {}
    inertia, best_inertia = {}, {}
    g_mean = np.mean(x_org, axis=0)
    
    x_zscr_std, x_rng_std = standardizer(x=x_org)
        
#     x = x_org
    
    for i in range(n_repeats):
        clusters[i] = {}
        cluster_means[i] = {}
        differences[i] = {}
        rel_differences[i] = {}
        indices[i] = {}
        inertia[i] = {}
        
        # instantiate KMeans Alg. object
        km = KMeans(n_clusters=n_clusters, init='random', n_init=1, max_iter=500,
                    tol=1e-4, random_state=i, algorithm='full', )  # verbose=1
        if std_method.lower() == 'r' or \
        std_method.lower() == 'rng' or \
        std_method.lower() == 'range':
            km.fit(x_rng_std)  # Compute k-means by calling fit method 
            
        else:
            km.fit(x_zscr_std)  # Compute k-means by calling fit method 
        
        # Store the computation results per each initilization
        for k in range(n_clusters):
            clusters[i][k] = x_org[np.where(km.labels_==k)]
            indices[i][k] = np.where(km.labels_==k)[0]
            
        inertia[i] = km.inertia_
        
        for k in range(n_clusters):
            cluster_means[i][k] = np.mean(clusters[i][k], axis=0)    
            differences[i][k] = np.subtract(cluster_means[i][k], g_mean)
            rel_differences[i][k] = 100*(np.divide(
                np.subtract(cluster_means[i][k], g_mean), g_mean)
                                        )
        # to chose the best clustering results regarding the inertia
        if i == 0 :
            tmp_inertia = km.inertia_
            delta = 0
        if i != 0:
            delta = tmp_inertia - km.inertia_
        if delta >= 0:
            tmp_inertia = km.inertia_    
            for k in range(n_clusters):
                best_clusters[k] = x_org[np.where(km.labels_==k)]
                best_indices[k] = np.where(km.labels_==k)[0]
            for k in range(n_clusters):
                best_cluster_means[k] = np.mean(best_clusters[k], axis=0)
                best_differences[k] = np.subtract(best_cluster_means[k], g_mean)
                best_rel_differences[k] = 100*(np.divide(
                    np.subtract(best_cluster_means[k], g_mean), g_mean)
                                              )
            best_inertia = km.inertia_
    
    return  clusters, best_clusters, indices, \
best_indices, cluster_means, best_cluster_means, \
differences, rel_differences, best_differences, \
best_rel_differences , inertia, best_inertia

In [6]:
def demonstrate_best_results(x, features, clusters, indices,
                             cluster_means, differences, rel_differences):
    for cluster, result in clusters.items():
        print(str(cluster+1), "\t #",
              len(indices[cluster] ), 
              "\t", cluster_means[cluster])
    print("S", "\t #", x.shape[0], "\t",
          np.mean(x, axis=0))
        

# Clustering

In [7]:
n_clusters =  9
n_repeats = 10
clusters_1, best_clusters_1, indices_1, best_indices_1,\
cluster_means_1, best_cluster_means_1,\
differences_1, rel_differences_1, \
best_differences_1, best_rel_differences_1, \
inertia_1, best_intertia_1 = apply_kmeans(x_org=data, 
                                          n_clusters=n_clusters,
                                          n_repeats=n_repeats, 
                                          std_method='z')

In [8]:
features = [feature.strip().split(",")[0] for feature in features_]

demonstrate_best_results(x=data, features=features,
                         clusters=best_clusters_1,
                         indices=best_indices_1, 
                         cluster_means=best_cluster_means_1, 
                         differences=best_differences_1, 
                         rel_differences=best_rel_differences_1)

1 	 # 2 	 [191.75  169.856  22.88  999.787  72.636]
2 	 # 6 	 [296.576 280.274  16.275 899.14  423.349]
3 	 # 22 	 [235.869 222.206  13.694 796.813 206.835]
4 	 # 3 	 [97.154 90.165  6.99  44.633 97.967]
5 	 # 8 	 [184.584 173.772  10.798 970.122 264.067]
6 	 # 1 	 [ 339.586  290.917   48.688 1494.369  609.941]
7 	 # 12 	 [144.715 136.715   7.983 717.38  135.421]
8 	 # 10 	 [ 66.644  62.352   4.266 669.735  61.791]
9 	 # 27 	 [191.877 180.306  11.576 711.05  151.047]
S 	 # 91 	 [187.291 175.757  11.557 756.239 182.125]


To Select a feature and two clusters
We select feature number 0, 1st column, clusters number 5 and 9.

We are going to validate the mean of feature 0

And compare within cluster means C5 and C9

And compare C9 and the grand mean

In [9]:
feature_n = 0

In [10]:
def compute_confidence(resampled_means):
    
    """
        Compute the confidence interval
        Arguments: 
            resampled_means data, numpy array.
        Returns:
            bootstrap means, bootstrap std, and pivotal and non pivotal boundries
    """
    
    # pivotal method:
    boots_mean = np.mean(resampled_means)
    boots_std = np.std(resampled_means)
    
    
    lbp = boots_mean - 1.96*boots_std  # left bound pivotal
    rbp = boots_mean + 1.96*boots_std  # right bound pivotal

    # Non-pivotal:
    lower_bound_index = int((len(resampled_means))*.025)
    higher_bound_index = int((len(resampled_means))*.975)
    resampled_mean_sorted = sorted(resampled_means,)

    lbn = resampled_mean_sorted[lower_bound_index]  # left bound non pivotal
    rbn = resampled_mean_sorted[higher_bound_index]  # right bound non pivotal
    
    return boots_mean, boots_std, lbp, rbp, lbn, rbn

In [11]:
def bootstrapping(x, x_idx_1, x_idx_2, n_resample, n_bootstrap):
    
    """
        x, a 1-D array, representing the random samples.
        x_idx, a 1-D array, representing the random samples indices:
            e.g 1st cluster indices (membership vector) or
            a range from 0 to number of random samples.
        n_resample, an int., representing the random sampling size.
        n_bootstrap, an int, representing the number of 
            repeats in the bootstrapping procedure.

        Returns x_resampled_means, x_resampled_stds,
                boots_mean, boots_std,
                lbp, rbp, lbn, rbn, 
        where:
            x_resampled_means: list of means of resampled data(x) of the length n_resample;
            x_resampled_stds: list of standard deviations of resampled data(x);
                of the length n_resample,
            boots_mean: floating number, representing bootstrap mean --i.e mean of means)
            boots_std: floating number, representing bootstrap std --i.e std of stds.
    """

    np.random.seed(43)  # for the sake of reproducibility
    x_resampled_means, x_resampled_stds = [], []  # grand mean
    x_resampled_means_1, x_resampled_stds_1 = [], []  # 1st subset 
    x_resampled_means_2, x_resampled_stds_2 = [], []  # 2nd subset

    for _ in range(n_bootstrap):
        
        # Grand mean:
        list_to_choose = range(0, n_resample, 1)
        resampled_idx = np.random.choice(list_to_choose,
                                         size=n_resample, 
                                         replace=True,)
        
        x_resampled = x[resampled_idx]
        x_resampled_means.append(np.mean(x_resampled))
        x_resampled_stds.append(np.std(x_resampled))  # /np.sqrt(len(x)))
        
        # 1st subset:
        idx_1 = [i for i in resampled_idx if i in x_idx_1]
        x_resampled_1 = x[idx_1]
        x_resampled_means_1.append(np.mean(x_resampled_1))
        x_resampled_stds_1.append(np.std(x_resampled_1))  # /np.sqrt(len(x)))
        
        # 2nd subset:
        idx_2 = [i for i in resampled_idx if i in x_idx_2]
        x_resampled_2 = x[idx_2]
        x_resampled_means_2.append(np.mean(x_resampled_2))
        x_resampled_stds_2.append(np.std(x_resampled_2))  # /np.sqrt(len(x)))
        
    x_resampled_means = np.asarray(x_resampled_means)
    x_resampled_means_1 = np.asarray(x_resampled_means_1)
    x_resampled_means_2 = np.asarray(x_resampled_means_2)
    
    # Grand mean:
    boots_mean, boots_std, lbp, rbp, lbn, rbn = compute_confidence(
        resampled_means=x_resampled_means)
    
    # Subtraction of 1st cluster from grand mean:
    d1g = np.subtract(x_resampled_means_1, x_resampled_means)
    boots_mean_1g, boots_std_1g, lbp_1g, rbp_1g, lbn_1g, rbn_1g = compute_confidence(
        resampled_means=d1g)
    
    # Subtraction of 1st cluster from 2nd cluster:
    d12 = np.subtract(x_resampled_means_1, x_resampled_means_2)
    boots_mean_12, boots_std_12, lbp_12, rbp_12, lbn_12, rbn_12 = compute_confidence(
        resampled_means=d12)
    
    # x_resampled_means, x_resampled_stds, boots_mean, boots_std, lbp, rbp, lbn, rbn
    
    return ((lbp, rbp, lbn, rbn), 
            (lbp_1g, rbp_1g, lbn_1g, rbn_1g), 
            (lbp_12, rbp_12, lbn_12, rbn_12))

In [12]:
n_resample = data.shape[0]
n_bootstrap = 5000

In [13]:
((lbp, rbp, lbn, rbn), \
 (lbp_1g, rbp_1g, lbn_1g, rbn_1g), \
 (lbp_12, rbp_12, lbn_12, rbn_12)) = bootstrapping(x=data[:, feature_n],
                                                   x_idx_1=best_indices_1[4],
                                                   x_idx_2=best_indices_1[8],
                                                   n_resample=n_resample,
                                                   n_bootstrap=n_bootstrap)

### Presentation

##### 5000 trials means

In [14]:
print("\t \t Left Bound \t \t  Right Bound ")
print("Pivotal    :", lbp,
      "\t", rbp, 
      "\n"
      "Nonpivotal :", lbn, 
      "rbn\t", rbn)

	 	 Left Bound 	 	  Right Bound 
Pivotal    : 173.85886641735394 	 200.56416418923945 
Nonpivotal : 173.91437362637365 rbn	 200.4796923076923


##### subtraction of first subset from grand mean

In [15]:
print("\t \t Left Bound \t \t  Right Bound ")
print("Pivotal    :", lbp_1g,
      "\t", rbp_1g, 
      "\n"
      "Nonpivotal :", lbn_1g, 
      "rbn\t", rbn_1g)

	 	 Left Bound 	 	  Right Bound 
Pivotal    : -21.67231900602535 	 16.393458466086546 
Nonpivotal : -22.057054945055 rbn	 15.984296703296735


##### subtraction of first subset from  second subset

In [16]:
print("\t \t Left Bound \t \t  Right Bound ")
print("Pivotal    :", lbp_12,
      "\t", rbp_12, 
      "\n"
      "Nonpivotal :", lbn_12, 
      "rbn\t", rbn_12)

	 	 Left Bound 	 	  Right Bound 
Pivotal    : -22.930068781767716 	 8.349173721077575 
Nonpivotal : -23.057066666666685 rbn	 8.420068376068343


The mean of one cluster, for the aformentioned feature, is not greater than another.